# Analyzing Chicago crime data using Apache Spark.
Data Source https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2/data
We will learn little bit from Spark as well.

Import SparkSession
As of Spark 2.0, we only need to create SparkSession. We can create sparkContext from SparkSession.

In [1]:
from pyspark.sql import SparkSession

Build SparkSession using spark standalone mode.
Well, format is staright forward. Only thing to note here is the spark master server url. We can use local, mesos and yarn as well.

In [2]:
spark = SparkSession.builder.master("spark://10.142.0.2:7077").appName('Analyze Chicago crime data').getOrCreate()

Read crime data as parquet file from hdfs location. Use my other notebook csvToParquet.ipynb to convert this crime data from csv to parquet format.

spark.read provides many data source apis like csv, avro, parquet,json etc.

In [3]:
crimeData = spark.read.parquet("hdfs://localhost:9000/public/data/crime/parquet")

We are going to use crimeData dataframe a lot. So it is better to cache it in-memory. Simply call call cache() method on this dataframe.

In [4]:
crimeData.cache()

DataFrame[ID: int, CaseNumber: string, Block: string, IUCR: string, PrimaryType: string, Description: string, LocationDescription: string, Arrest: boolean, Domestic: boolean, Beat: int, District: int, Ward: int, CommunityArea: int, FBICode: string, XCoordinate: int, YCoordinate: int, Latitude: double, Longitude: double, ReportedTime: timestamp, UpdatedTime: timestamp]

Show data. For this we have method called show(). This is the action operation. By default show will display 20 rows only. We can give number of rows as argument as well like show(30) which will show 30 rows.

In [5]:
crimeData.show()

+--------+----------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+------------+-------------+-------------------+-------------------+
|      ID|CaseNumber|               Block|IUCR|         PrimaryType|         Description| LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|    Latitude|    Longitude|       ReportedTime|        UpdatedTime|
+--------+----------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+------------+-------------+-------------------+-------------------+
|10000092|  HY189866|     047XX W OHIO ST|041A|             BATTERY| AGGRAVATED: HANDGUN|              STREET| false|   false|1111|      11|  28|           25|    04B|    1144606|    1903566|41.891398861|-87.744384567|2015-

Run count on your dataframe.

In [6]:
crimeData.count()

6586188

Repartition data to 8 partition because I am using 4 cores machine. It is good to have partition in multiple of your number of cores for good performance. But there are other factors as well while deciding on a number.

repartition and coalesce are two operations to manage number of partitions. coalesce should be used only when you want to decrease the total number of partitions while repartition can be used any time but repartition is costly operation. So learn about repartiton and coalesce in more detail.

In [7]:
crimeData.repartition(8)

DataFrame[ID: int, CaseNumber: string, Block: string, IUCR: string, PrimaryType: string, Description: string, LocationDescription: string, Arrest: boolean, Domestic: boolean, Beat: int, District: int, Ward: int, CommunityArea: int, FBICode: string, XCoordinate: int, YCoordinate: int, Latitude: double, Longitude: double, ReportedTime: timestamp, UpdatedTime: timestamp]

Import functions like max, min, count etc.

In [8]:
from pyspark.sql import functions as F

For 2018 we don't have complete data as 2018 is the current year so I am removing records of 2018 from my data.

Spark dataframe API gives us filter() operation for filtering data based on some condition. Also year(timestamp) will return year of the timestamp.

In [9]:
crimeData = crimeData.filter(F.year('ReportedTime') != 2018)

First we will calculate how many crimes have been reported in each year.

For this we need to group our elements by year and then count. 

Here we have,

groupBy() which will group elements based on one or more fields passed to it.

alias() will give an alias to the column.

count() will simply count.

orderBy() will order the data based on one or more fields passed to it. By default it is in ascending order.

In [10]:
crimeData.groupBy(F.year('ReportedTime').alias('Year')).count().orderBy('Year').show()

+----+------+
|Year| count|
+----+------+
|2001|485742|
|2002|486749|
|2003|475929|
|2004|469371|
|2005|453693|
|2006|448086|
|2007|436981|
|2008|427013|
|2009|392628|
|2010|370262|
|2011|351703|
|2012|335855|
|2013|306915|
|2014|274992|
|2015|263694|
|2016|268504|
|2017|266967|
+----+------+



If you see the header then one of the most important field is the primary crime type. Here we are calculating how many primary crime type are there in the dataset.

select() will select only the mentioned fields from the dataframe. It is like select in the SQL.

distincit() will remove the select only the one row from duplicate rows.

In [11]:
crimeData.select('PrimaryType').distinct().count()

35

Lets find the name of each type. In case you are wondering why I am using show like show(35,False) because 35 is the number of distinct primary crime type and False because I want spark to adjust column width to longest string name.

In [12]:
crimeData.select('PrimaryType').distinct().show(35, False)

+---------------------------------+
|PrimaryType                      |
+---------------------------------+
|OFFENSE INVOLVING CHILDREN       |
|STALKING                         |
|PUBLIC PEACE VIOLATION           |
|OBSCENITY                        |
|NON-CRIMINAL (SUBJECT SPECIFIED) |
|ARSON                            |
|DOMESTIC VIOLENCE                |
|GAMBLING                         |
|CRIMINAL TRESPASS                |
|ASSAULT                          |
|NON - CRIMINAL                   |
|LIQUOR LAW VIOLATION             |
|MOTOR VEHICLE THEFT              |
|THEFT                            |
|BATTERY                          |
|ROBBERY                          |
|HOMICIDE                         |
|RITUALISM                        |
|PUBLIC INDECENCY                 |
|CRIM SEXUAL ASSAULT              |
|HUMAN TRAFFICKING                |
|INTIMIDATION                     |
|PROSTITUTION                     |
|DECEPTIVE PRACTICE               |
|CONCEALED CARRY LICENSE VIO

Another field that I found interesting is Location Description. So I found how many crimes were committed based on each location. We can see in the result that Steert, residence are the most unsafe places.

Here we want the list in decending order of total number of cases and hence we are passing ascending=False to orderBy().

In [13]:
crimeData.groupBy('LocationDescription').count().orderBy('count',ascending=False).show(20, False)

+------------------------------+-------+
|LocationDescription           |count  |
+------------------------------+-------+
|STREET                        |1719290|
|RESIDENCE                     |1103634|
|APARTMENT                     |667663 |
|SIDEWALK                      |646701 |
|OTHER                         |247040 |
|PARKING LOT/GARAGE(NON.RESID.)|187034 |
|ALLEY                         |146470 |
|SCHOOL, PUBLIC, BUILDING      |139295 |
|RESIDENCE-GARAGE              |128145 |
|RESIDENCE PORCH/HALLWAY       |114000 |
|SMALL RETAIL STORE            |112943 |
|VEHICLE NON-COMMERCIAL        |103914 |
|RESTAURANT                    |99267  |
|GROCERY FOOD STORE            |84291  |
|DEPARTMENT STORE              |79655  |
|GAS STATION                   |68741  |
|RESIDENTIAL YARD (FRONT/BACK) |64893  |
|CHA PARKING LOT/GROUNDS       |54984  |
|PARK PROPERTY                 |50659  |
|COMMERCIAL / BUSINESS OFFICE  |47681  |
+------------------------------+-------+
only showing top

But what kind of crimes were commmitted on these location. So next thing is to find number of crimes per location per crime type.

Simply pass PrimaryType as second argument to groupBy()

In [14]:
crimeData.groupBy('LocationDescription','PrimaryType').count().orderBy('count',ascending=False).show(20, False)

+------------------------------+-------------------+------+
|LocationDescription           |PrimaryType        |count |
+------------------------------+-------------------+------+
|STREET                        |THEFT              |375330|
|STREET                        |CRIMINAL DAMAGE    |270028|
|STREET                        |NARCOTICS          |249071|
|RESIDENCE                     |BATTERY            |248682|
|STREET                        |MOTOR VEHICLE THEFT|241949|
|APARTMENT                     |BATTERY            |232683|
|SIDEWALK                      |NARCOTICS          |219514|
|STREET                        |BATTERY            |212732|
|RESIDENCE                     |OTHER OFFENSE      |187422|
|SIDEWALK                      |BATTERY            |176715|
|RESIDENCE                     |THEFT              |144477|
|RESIDENCE                     |CRIMINAL DAMAGE    |141730|
|RESIDENCE                     |BURGLARY           |129608|
|APARTMENT                     |BURGLARY

Now we need to find what crime types are most common.

Here we will use agg() instead of count(). agg() will let us do many operations like finding maximum, minimum, sum and more.

Syntax: agg(max()) or agg(sum())

In [15]:
topPrimaryType = crimeData.groupBy('PrimaryType').agg(F.count('*').alias('Total')).orderBy('Total',ascending=False)
topPrimaryType.show(10, False)

+-------------------+-------+
|PrimaryType        |Total  |
+-------------------+-------+
|THEFT              |1361972|
|BATTERY            |1188562|
|CRIMINAL DAMAGE    |747161 |
|NARCOTICS          |699598 |
|OTHER OFFENSE      |403641 |
|ASSAULT            |400642 |
|BURGLARY           |377828 |
|MOTOR VEHICLE THEFT|305659 |
|ROBBERY            |247260 |
|DECEPTIVE PRACTICE |245842 |
+-------------------+-------+
only showing top 10 rows



But I wanted to see how many arrests have been made in each crime type.

For this we will filter data where Arrest is true and then run groupBy on PrimaryType to get the count.

In [16]:
topPrimaryTypeArrest = crimeData.filter(crimeData['Arrest'] == True).groupBy('PrimaryType').agg(F.count('*').alias('Arrest')).orderBy('Arrest',ascending=False)
topPrimaryTypeArrest.show(10, False)

+------------------+------+
|PrimaryType       |Arrest|
+------------------+------+
|NARCOTICS         |695378|
|BATTERY           |271890|
|THEFT             |163909|
|CRIMINAL TRESPASS |138302|
|ASSAULT           |93883 |
|OTHER OFFENSE     |71578 |
|PROSTITUTION      |67430 |
|CRIMINAL DAMAGE   |52980 |
|WEAPONS VIOLATION |52916 |
|DECEPTIVE PRACTICE|43348 |
+------------------+------+
only showing top 10 rows



Well we can understand this. Narcotics is the highest priority from the Chicago crime department.

But we can't really say that Narcotics is the priority untill we find how many cases were reported and how many arrests were made.

In my previous two cells, first I find the total number of cases for each crime type and then I find total number of cases where arrests were made for each crime type. For getting the percentage of arrests we need both of these columns. So we will simply join our two dataframes on 'PrimaryType' and the get the percenatge.

col() will let me perform operations on values of individual rows.

In [17]:
topPrimaryType.join(topPrimaryTypeArrest, topPrimaryType.PrimaryType ==topPrimaryTypeArrest.PrimaryType).select(topPrimaryTypeArrest['PrimaryType'],'Total','Arrest', (F.col('Arrest')/F.col('Total')*100).alias('ArrestPercentage')).orderBy('ArrestPercentage',ascending=0).show(10,False)

+---------------------------------+------+------+-----------------+
|PrimaryType                      |Total |Arrest|ArrestPercentage |
+---------------------------------+------+------+-----------------+
|DOMESTIC VIOLENCE                |1     |1     |100.0            |
|PROSTITUTION                     |67697 |67430 |99.60559552121954|
|NARCOTICS                        |699598|695378|99.39679644595897|
|PUBLIC INDECENCY                 |150   |149   |99.33333333333333|
|GAMBLING                         |14226 |14122 |99.26894418670041|
|LIQUOR LAW VIOLATION             |13815 |13690 |99.09518639160333|
|INTERFERENCE WITH PUBLIC OFFICER |13989 |12792 |91.44327686039031|
|CONCEALED CARRY LICENSE VIOLATION|154   |139   |90.25974025974025|
|OBSCENITY                        |503   |416   |82.7037773359841 |
|WEAPONS VIOLATION                |65924 |52916 |80.26818760997513|
+---------------------------------+------+------+-----------------+
only showing top 10 rows



Yeah. So it is prostitution and narcotics. And see theft is not in top 10 list. This makes sense as most thiefs run away before one can report crime. 

But what are the crime types where least number of arrest happens.

In [18]:
topPrimaryType.join(topPrimaryTypeArrest, topPrimaryType.PrimaryType ==topPrimaryTypeArrest.PrimaryType).select(topPrimaryTypeArrest['PrimaryType'],'Total','Arrest', (F.col('Arrest')/F.col('Total')*100).alias('ArrestPercentage')).orderBy('ArrestPercentage',ascending=1).show(10,False)

+-------------------+-------+------+------------------+
|PrimaryType        |Total  |Arrest|ArrestPercentage  |
+-------------------+-------+------+------------------+
|BURGLARY           |377828 |21716 |5.747588849952889 |
|NON-CRIMINAL       |131    |8     |6.106870229007633 |
|CRIMINAL DAMAGE    |747161 |52980 |7.090841197546446 |
|MOTOR VEHICLE THEFT|305659 |28202 |9.226621823666243 |
|ROBBERY            |247260 |24130 |9.758958181671115 |
|KIDNAPPING         |6516   |733   |11.249232658072437|
|THEFT              |1361972|163909|12.034682063948452|
|RITUALISM          |23     |3     |13.043478260869565|
|ARSON              |10834  |1425  |13.153036736200848|
|HUMAN TRAFFICKING  |36     |5     |13.88888888888889 |
+-------------------+-------+------+------------------+
only showing top 10 rows



Burglary, robbery, theft are there but what I don't like is to see kidnapping also. I think kidnapping should get more priority.

But whether Chicago or any other part of the world, crime related to sex are there in all places. I wanted to see what is the arrest percentage in crimes related to sex.

For this I will simply put filter before selecting in my query.

I am using like() here which just like SQL let me do pattern matching.

In [19]:
topPrimaryType.join(topPrimaryTypeArrest, topPrimaryType.PrimaryType ==topPrimaryTypeArrest.PrimaryType).filter(topPrimaryTypeArrest['PrimaryType'].like('%SEX%')).select(topPrimaryTypeArrest['PrimaryType'],'Total','Arrest', (F.col('Arrest')/F.col('Total')*100).alias('ArrestPercentage')).show(10,False)

+-------------------+-----+------+------------------+
|PrimaryType        |Total|Arrest|ArrestPercentage  |
+-------------------+-----+------+------------------+
|CRIM SEXUAL ASSAULT|25614|4110  |16.04591239166081 |
|SEX OFFENSE        |24073|7527  |31.267395006854155|
+-------------------+-----+------+------------------+



I don't have much knowledge about how law works in Chicago so I can't comment if this number is good or bad.

Next this is to find the blocks with maximum cases of crime.

In [20]:
topBlock = crimeData.groupBy('Block','PrimaryType').agg(F.count('*').alias('Total')).orderBy('Total',ascending=False)
topBlock.show(10, False)

+-------------------------+-------------+-----+
|Block                    |PrimaryType  |Total|
+-------------------------+-------------+-----+
|001XX N STATE ST         |THEFT        |8085 |
|076XX S CICERO AVE       |THEFT        |6677 |
|0000X N STATE ST         |THEFT        |6387 |
|008XX N MICHIGAN AVE     |THEFT        |6169 |
|100XX W OHARE ST         |THEFT        |5241 |
|100XX W OHARE ST         |OTHER OFFENSE|2568 |
|001XX W 87TH ST          |THEFT        |2408 |
|046XX W NORTH AVE        |THEFT        |2384 |
|033XX W FILLMORE ST      |NARCOTICS    |2351 |
|085XX S COTTAGE GROVE AVE|THEFT        |2225 |
+-------------------------+-------------+-----+
only showing top 10 rows



Again I want to see the arrest percentage.

In [21]:
topBlockArrest = crimeData.filter(crimeData['Arrest'] == True).groupBy('Block','PrimaryType').agg(F.count('*').alias('Arrest')).orderBy('Arrest',ascending=False)
topBlockArrest.show(10, False)

+--------------------+------------------+------+
|Block               |PrimaryType       |Arrest|
+--------------------+------------------+------+
|001XX N STATE ST    |THEFT             |5049  |
|0000X N STATE ST    |THEFT             |3851  |
|076XX S CICERO AVE  |THEFT             |3657  |
|008XX N MICHIGAN AVE|THEFT             |2719  |
|033XX W FILLMORE ST |NARCOTICS         |2333  |
|046XX W NORTH AVE   |THEFT             |1860  |
|040XX W LAKE ST     |DECEPTIVE PRACTICE|1714  |
|005XX E BROWNING AVE|NARCOTICS         |1695  |
|023XX S STATE ST    |CRIMINAL TRESPASS |1657  |
|008XX N STATE ST    |DECEPTIVE PRACTICE|1619  |
+--------------------+------------------+------+
only showing top 10 rows



In [22]:
topBlock.join(topBlockArrest, ['Block','PrimaryType']).filter(topBlock['Total'] > 1000).select('Block','PrimaryType','Total','Arrest', (F.col('Arrest')/F.col('Total')*100).alias('ArrestPercentage')).orderBy('ArrestPercentage',ascending=0).show(30,False)

+-------------------------+------------------+-----+------+-----------------+
|Block                    |PrimaryType       |Total|Arrest|ArrestPercentage |
+-------------------------+------------------+-----+------+-----------------+
|044XX S FEDERAL ST       |NARCOTICS         |1107 |1107  |100.0            |
|051XX W MADISON ST       |NARCOTICS         |1077 |1077  |100.0            |
|036XX S FEDERAL ST       |NARCOTICS         |1128 |1127  |99.91134751773049|
|039XX W GRENSHAW ST      |NARCOTICS         |1052 |1049  |99.7148288973384 |
|038XX W ROOSEVELT RD     |NARCOTICS         |1459 |1450  |99.38313913639479|
|005XX E BROWNING AVE     |NARCOTICS         |1706 |1695  |99.35521688159437|
|033XX W FILLMORE ST      |NARCOTICS         |2351 |2333  |99.23436835389197|
|023XX S STATE ST         |NARCOTICS         |1224 |1211  |98.93790849673204|
|029XX S STATE ST         |NARCOTICS         |1126 |1114  |98.93428063943162|
|027XX S DEARBORN ST      |NARCOTICS         |1061 |1042  |98.20

As expected, Narcotics.

In [23]:
topBlock.join(topBlockArrest, ['Block','PrimaryType']).filter(topBlock['Total'] > 1000).select('Block','PrimaryType','Total','Arrest', (F.col('Arrest')/F.col('Total')*100).alias('ArrestPercentage')).orderBy('ArrestPercentage',ascending=1).show(30,False)

+-----------------------------------+------------------+-----+------+------------------+
|Block                              |PrimaryType       |Total|Arrest|ArrestPercentage  |
+-----------------------------------+------------------+-----+------+------------------+
|100XX W OHARE ST                   |OTHER OFFENSE     |2568 |91    |3.543613707165109 |
|0000X N MICHIGAN AVE               |THEFT             |1024 |39    |3.80859375        |
|001XX E WACKER DR                  |THEFT             |1078 |47    |4.3599257884972165|
|002XX N MICHIGAN AVE               |THEFT             |1021 |64    |6.268364348677767 |
|051XX S PULASKI RD                 |THEFT             |1044 |66    |6.321839080459771 |
|057XX S CICERO AVE                 |THEFT             |1264 |83    |6.5664556962025316|
|100XX W OHARE ST                   |THEFT             |5241 |359   |6.849837817210457 |
|006XX E GRAND AVE                  |THEFT             |1326 |91    |6.862745098039216 |
|0000X E JACKSON BLVD

And of course a clear winner here as well.

But what blocks reported maximum cases related to sexual offence.

In [24]:
topBlock.filter(topBlock['PrimaryType'].like('%SEX%')).show()

+--------------------+-------------------+-----+
|               Block|        PrimaryType|Total|
+--------------------+-------------------+-----+
|  044XX N SIMONDS DR|        SEX OFFENSE|   76|
|    004XX S STATE ST|        SEX OFFENSE|   58|
|031XX W DOUGLAS BLVD|        SEX OFFENSE|   49|
|   012XX S DAMEN AVE|        SEX OFFENSE|   44|
|011XX S HAMILTON AVE|        SEX OFFENSE|   35|
|    100XX W OHARE ST|        SEX OFFENSE|   35|
|    002XX E HURON ST|CRIM SEXUAL ASSAULT|   33|
|063XX S DR MARTIN...|CRIM SEXUAL ASSAULT|   30|
| 009XX W BELMONT AVE|        SEX OFFENSE|   30|
|003XX W MONTROSE ...|        SEX OFFENSE|   27|
|    001XX N STATE ST|        SEX OFFENSE|   27|
|075XX S STONY ISL...|CRIM SEXUAL ASSAULT|   27|
|  046XX N SIMONDS DR|        SEX OFFENSE|   26|
|   048XX N MARINE DR|CRIM SEXUAL ASSAULT|   25|
|     0000X W 95TH ST|        SEX OFFENSE|   25|
|     003XX W 64TH ST|CRIM SEXUAL ASSAULT|   21|
| 0000X W DIVISION ST|CRIM SEXUAL ASSAULT|   21|
|0000X E ROOSEVELT R

Let me set spark.sql.shuffle.partitions to 32 which is 200 by default. This number decides how many tasks will be created. You don't want to create 200 task in each use case. For this I think 32 is fine.

In [25]:
spark.conf.set('spark.sql.shuffle.partitions','32')

Here I want to see which block made maximum arrests in each of crimes related to sexual offence.

In [26]:
topBlock.join(topBlockArrest, ['Block','PrimaryType']).filter(topBlock['PrimaryType'].like('%SEX%')).filter(topBlock['Total']>15).select('Block','PrimaryType','Total','Arrest', (F.col('Arrest')/F.col('Total')*100).alias('ArrestPercentage')).orderBy('ArrestPercentage',ascending=0).show(30,False)

+-----------------------------------+-------------------+-----+------+------------------+
|Block                              |PrimaryType        |Total|Arrest|ArrestPercentage  |
+-----------------------------------+-------------------+-----+------+------------------+
|046XX N SIMONDS DR                 |SEX OFFENSE        |26   |25    |96.15384615384616 |
|044XX N SIMONDS DR                 |SEX OFFENSE        |76   |72    |94.73684210526315 |
|002XX W MONTROSE HARBOR DR         |SEX OFFENSE        |17   |16    |94.11764705882352 |
|052XX N CICERO AVE                 |SEX OFFENSE        |17   |15    |88.23529411764706 |
|004XX S STATE ST                   |SEX OFFENSE        |58   |49    |84.48275862068965 |
|003XX W MONTROSE HARBOR DR         |SEX OFFENSE        |27   |19    |70.37037037037037 |
|009XX W BELMONT AVE                |SEX OFFENSE        |30   |19    |63.33333333333333 |
|001XX N STATE ST                   |SEX OFFENSE        |27   |17    |62.96296296296296 |
|076XX S C

Next thing I want to find out is the months where maximum crime is reported over the years and see if there is something comman.

Import Window from pyspark.sql

The reason we are using window here because we need to find month in each year where maximum number of crimes were reported.

What I am going to do here is to group my data according to month and year and then count. Then I will find the rank of each month per year based on the maximum count. We will use rank() which is a windowing function.

There must a different and/or better solution for this but as of now I know only this.

In [27]:
from pyspark.sql import Window

In [28]:
monthlyCount=crimeData.groupBy(F.month('ReportedTime').alias('Month'),F.year('ReportedTime').alias('Year')).agg(F.count('*').alias('Total')).select('Year','Month','Total',(F.rank().over(Window.partitionBy('Year').orderBy(F.col('Total').desc()))).alias('Rank'))
monthlyCount.filter(monthlyCount['Rank'] ==1).orderBy('Year').show()

+----+-----+-----+----+
|Year|Month|Total|Rank|
+----+-----+-----+----+
|2001|    7|44690|   1|
|2002|    7|46013|   1|
|2003|    8|44264|   1|
|2004|    7|43235|   1|
|2005|    7|41804|   1|
|2006|    7|41542|   1|
|2007|    7|40986|   1|
|2008|    8|40536|   1|
|2009|    8|35819|   1|
|2010|    8|34145|   1|
|2011|    7|33242|   1|
|2012|    7|31958|   1|
|2013|    8|28630|   1|
|2014|    7|26511|   1|
|2015|    8|24743|   1|
|2016|    8|24823|   1|
|2017|    7|24723|   1|
+----+-----+-----+----+



In [29]:
monthlyCount.filter(monthlyCount['Rank'] <=2).orderBy('Year','Rank').show(40)

+----+-----+-----+----+
|Year|Month|Total|Rank|
+----+-----+-----+----+
|2001|    7|44690|   1|
|2001|    8|44032|   2|
|2002|    7|46013|   1|
|2002|    8|44210|   2|
|2003|    8|44264|   1|
|2003|    7|43415|   2|
|2004|    7|43235|   1|
|2004|    8|43044|   2|
|2005|    7|41804|   1|
|2005|    8|41541|   2|
|2006|    7|41542|   1|
|2006|    8|40493|   2|
|2007|    7|40986|   1|
|2007|    5|40084|   2|
|2008|    8|40536|   1|
|2008|    7|40483|   2|
|2009|    8|35819|   1|
|2009|    7|35668|   2|
|2010|    8|34145|   1|
|2010|    7|33507|   2|
|2011|    7|33242|   1|
|2011|    8|32576|   2|
|2012|    7|31958|   1|
|2012|    6|31066|   2|
|2013|    8|28630|   1|
|2013|    7|28605|   2|
|2014|    7|26511|   1|
|2014|    8|25837|   2|
|2015|    8|24743|   1|
|2015|    7|24158|   2|
|2016|    8|24823|   1|
|2016|    7|24786|   2|
|2017|    7|24723|   1|
|2017|    8|24591|   2|
+----+-----+-----+----+



So month 7 and 8 are clear winner here.

# That's it for this notebook. I think we have covered many basic dataframe APIs. Of course there are more APIs, but once we know that basic we can play with others on our own.

## Note:
I am a newbie in apache spark. So this is what I have learned so far. My focus here was to show how we can use datafram APIs rather than writing optimized queries.